In [2]:
from subprocess import Popen
from getpass import getpass
import platform
import os
import shutil

In [3]:
homeDir = os.path.expanduser("~") + os.sep

# Create .urs_cookies and .dodsrc files
with open(homeDir + '.urs_cookies', 'w') as file:
    file.write('')
    file.close()
with open(homeDir + '.dodsrc', 'w') as file:
    file.write('HTTP.COOKIEJAR={}.urs_cookies\n'.format(homeDir))
    file.write('HTTP.NETRC={}.netrc'.format(homeDir))
    file.close()

print('Saved .urs_cookies and .dodsrc to:', homeDir)

# Copy dodsrc to working directory in Windows
if platform.system() == "Windows":  
    shutil.copy2(homeDir + '.dodsrc', os.getcwd())
    print('Copied .dodsrc to:', os.getcwd())

Saved .urs_cookies and .dodsrc to: /Users/armankhan/


In [4]:
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

homeDir = os.path.expanduser("~") + os.sep

with open(homeDir + '.netrc', 'w') as file:
    file.write('machine {} login {} password {}'.format(urs, getpass(prompt=prompts[0]), getpass(prompt=prompts[1])))
    file.close()

print('Saved .netrc to:', homeDir)

# Set appropriate permissions for Linux/macOS
if platform.system() != "Windows":
    Popen('chmod og-rw ~/.netrc', shell=True)

Saved .netrc to: /Users/armankhan/


In [6]:
import earthaccess

auth = earthaccess.login(strategy="interactive", persist=True)

In [7]:
import requests
from base64 import b64encode
from getpass import getpass
import os

# Earthdata Login URL for obtaining the token, and creating one if it doesn't exist
url = 'https://urs.earthdata.nasa.gov/api/users/find_or_create_token'

# Earthdata Login credential prompts
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Get credentials from user input
username = getpass(prompt=prompts[0])
password = getpass(prompt=prompts[1])

# Encode credentials using Base64
credentials = b64encode(f"{username}:{password}".encode('utf-8')).decode('utf-8')

# Headers with the Basic Authorization
headers = {
    'Authorization': f'Basic {credentials}'
}

# Make the POST request to get the token
response = requests.post(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON to get the token
    token_info = response.json()
    token = token_info.get("access_token")
    print("Token retrieved successfully")

    # Define the path for the .edl_token file in the home directory
    token_file_path = os.path.join(os.path.expanduser("~"), ".edl_token")

    # Write the token to the .edl_token file
    with open(token_file_path, 'w') as token_file:
        token_file.write(token)

    print(f"Token saved to {token_file_path}")

else:
    print("Failed to retrieve token:", response.text)

Token retrieved successfully
Token saved to /Users/armankhan/.edl_token


In [8]:
print(token_file_path)

/Users/armankhan/.edl_token
